In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

data = pd.read_csv('customer_data.csv')

print("Dataset Preview:")
display(data.head())

print("\nMissing Values:")
print(data.isnull().sum())

print("\nDataset Statistics:")
display(data.describe())

plt.figure(figsize=(6, 4))
sns.countplot(x='Gender', data=data, palette='Set2')
plt.title('Gender Distribution')
plt.show()

plt.figure(figsize=(8, 5))
sns.histplot(data['Age'], bins=20, kde=True, color='skyblue')
plt.title('Age Distribution')
plt.show()

plt.figure(figsize=(8, 5))
sns.scatterplot(x='Annual Income (k$)', y='Spending Score (1-100)', hue='Gender', data=data)
plt.title('Annual Income vs Spending Score')
plt.show()

features = ['Annual Income (k$)', 'Spending Score (1-100)']
X = data[features]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

inertia = []
K = range(1, 11)
for k in K:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(X_scaled)
    inertia.append(kmeans.inertia_)

plt.figure(figsize=(8, 5))
plt.plot(K, inertia, 'bo-')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Inertia')
plt.title('Elbow Method For Optimal k')
plt.show()

optimal_k = 5
kmeans = KMeans(n_clusters=optimal_k, random_state=42)
data['Cluster'] = kmeans.fit_predict(X_scaled)

plt.figure(figsize=(8, 5))
sns.scatterplot(x='Annual Income (k$)', y='Spending Score (1-100)',
                hue='Cluster', palette='Set1', data=data, s=100)
plt.title('Customer Segments (K-Means Clustering)')
plt.show()

pca = PCA(2)
pca_components = pca.fit_transform(X_scaled)

plt.figure(figsize=(8, 5))
sns.scatterplot(x=pca_components[:, 0], y=pca_components[:, 1],
                hue=data['Cluster'], palette='Set1', s=100)
plt.title('Customer Segments in 2D (PCA)')
plt.show()

cluster_summary = data.groupby('Cluster')[features].mean().round(2)
print("Cluster Summary:")
display(cluster_summary)

plt.figure(figsize=(8, 5))
sns.heatmap(cluster_summary, annot=True, cmap='coolwarm')
plt.title('Cluster Centers Heatmap')
plt.show()

for cluster in cluster_summary.index:
    income = cluster_summary.loc[cluster, 'Annual Income (k$)']
    score = cluster_summary.loc[cluster, 'Spending Score (1-100)']
    print(f"\nCluster {cluster} (Income: {income}k$, Score: {score}):")
    if score > 70:
        print("Target with premium offers and loyalty programs.")
    elif score > 40:
        print("Moderate engagement - seasonal discounts.")
    else:
        print("Low engagement - consider awareness campaigns.")
